In [1]:
%load_ext autoreload
%autoreload 2

from read_excel import *
from compute_state import *
from collections import namedtuple

In [44]:
# Example usage:
file_path = 'private/ordenes.xlsx'  # Set the filename to 'ordenes.xlsx'
df = read_excel_to_dataframe(file_path)
df = merge_date_time_columns(df)
print(df.loc[0])

Operacion           Suscripción Desde Cuenta Balanz
Estado                                    Ejecutada
id Orden                                   41387004
Ticker                                        BCMMA
Moneda                                        Pesos
Fecha                           2024-04-17 14:15:11
Cantidad                                       -1.0
Precio                                         -1.0
Monto                                     314704.13
Precio Operado                             6.300322
Cantidad Operada                       49950.483483
Name: 0, dtype: object


In [47]:
## Example usage:
# Create a Balance tuple
balance = Balance(ARS=0, USD=0)

# Create a State tuple containing the Balance tuple
current_state = State(balance=balance)

# Iterate through all rows of the DataFrame
for index, row in df.iterrows():
    
    # Call the update_state function for each row
    next_state = update_state(row, current_state)
    
    # Update the current state to the next state
    current_state = next_state

current_state

On 2024-04-17 14:15:11 USD price: Referencia    1021.16
Name: 2024-04-17, dtype: float64, ARS: 314704.13, USD: Referencia    308.182978
Name: 2024-04-17, dtype: float64
On 2024-04-15 10:23:03 USD price: Referencia    1021.55
Name: 2024-04-15, dtype: float64, ARS: -467073.42, USD: Referencia   -457.220322
Name: 2024-04-15, dtype: float64
On 2024-04-09 14:52:39 USD price: Referencia    1000.69
Name: 2024-04-09, dtype: float64, ARS: -210817.11, USD: Referencia   -210.671746
Name: 2024-04-09, dtype: float64
On 2024-03-27 15:30:43 USD price: Referencia    1016.09
Name: 2024-03-27, dtype: float64, ARS: -50000.0, USD: Referencia   -49.208239
Name: 2024-03-27, dtype: float64
On 2024-03-25 12:49:29 USD price: Referencia    1012.94
Name: 2024-03-25, dtype: float64, ARS: -19931.23, USD: Referencia   -19.676615
Name: 2024-03-25, dtype: float64
On 2024-03-22 16:53:03 USD price: Referencia    1036.16
Name: 2024-03-22, dtype: float64, ARS: 300000.0, USD: Referencia    289.530574
Name: 2024-03-22, dty

State(balance=Balance(ARS=2987990.45, USD=Referencia    7467.039436
dtype: float64))

In [60]:
# Define the lambda function
parse_referencia = lambda x: float(x.replace(',', '.').split(' ')[0])

# Read the CSV file, using the lambda function for parsing and parsing dates
df = pd.read_csv('cotizaciones_mep.csv', parse_dates=['Fecha'], dayfirst=True, converters={'Referencia': parse_referencia})

# Extract the date part from the 'Fecha' column and convert to Timestamp
df['Fecha'] = pd.to_datetime(df['Fecha']).dt.date

# If no exact match found, interpolate with nearest dates
df = df.set_index('Fecha')

print(df)

df.loc[pd.to_datetime("2024-03-05").date()][0]

            Referencia
Fecha                 
2024-05-03     1072.64
2024-05-02     1064.56
2024-05-01     1040.89
2024-04-30     1040.89
2024-04-29     1037.77
...                ...
2020-03-24       91.40
2020-03-23       91.40
2020-03-22       91.40
2020-03-21       91.40
2020-03-20       91.40

[1293 rows x 1 columns]


1016.08